# We select and load the model

In [ ]:
from transformers import pipeline
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image

pokemon= "Image of the Pokémon"
pipe = pipeline("image-classification", model="skshmjn/Pokemon-classifier-gen9-1025")
pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")


processor = AutoImageProcessor.from_pretrained("skshmjn/Pokemon-classifier-gen9-1025")
model = AutoModelForImageClassification.from_pretrained("skshmjn/Pokemon-classifier-gen9-1025")

img = Image.open(pokemon).convert("RGB")
inputs = processor(images=img, return_tensors='pt').to(model.device)


outputs = model(**inputs)
predicted_id = outputs.logits.argmax(-1).item()
predicted_pokemon = model.config.id2label[predicted_id]

# **We use the PokeAPI to obtain relevant and up-to-date information about the Pokémon.**

In [ ]:
import requests
import polars as pl

Api=requests.get(f"https://pokeapi.co/api/v2/pokemon/{predicted_pokemon.lower()}")
data=Api.json()

pokemon_info = pl.DataFrame({ "ID": data["id"],
    "Altura (m)": data["height"] / 10,
    "Peso (kg)": data["weight"] / 10,
    "Tipos": ", ".join([t["type"]["name"] for t in data["types"]]),
    "Habilidades": ", ".join([a["ability"]["name"] for a in data["abilities"]]),
    "HP": data["stats"][0]["base_stat"],
    "Ataque": data["stats"][1]["base_stat"],
    "Defensa": data["stats"][2]["base_stat"],
    "Ataque Especial": data["stats"][3]["base_stat"],
    "Defensa Especial": data["stats"][4]["base_stat"],
    "Velocidad": data["stats"][5]["base_stat"]})



print(f"The pokemon is #{predicted_id+1}\n{predicted_pokemon}\n\n{pokemon_info}")